In [ ]:
import pandas
import pandas as pd

from GA import GA
import optproblems.cec2005 as optproblems

# Test Functions
- F2: Shifted Schwefel’s Problem 1.2
    - Uni-modal without noise
- F4: Shifted Schwefel’s Problem 1.2 with Noise in Fitness
    - Uni-modal with noise
- F8: Shifted Rotated Ackley’s Function with Global Optimum on Bounds
    - Single function
    - Global optimum on bounds
    - Multi-modal
- F13: Shifted Expanded Griewank’s plus Rosenbrock’s Function (F8F2)
    - Expanded Function
    - Multi-modal
- F17: Rotated Version of Hybrid Composition Function with Noise in Fitness
    - Gaussian noise in fitness
    - Multi-modal
    - Huge number of local optima
    - In-deterministic
- F24: Rotated Hybrid Composition Function
    - Composite function
    - Huge number of local optima
    - Multi-modal

# 1st Iteration of Configurations - Experiment with GA with Different Hyper Parameters
First we explore how GA evaluates with a set of configurations using only lower numbers of generations

Hyper parameters

In [2]:
# F2 :   Unimodal without noise
# F4 :   Unimodal with noise
# F8 :   Single Function, Global Optimum on Bounds, Multi-modal
# F13:   Expanded Function, Multi-modal
# F17:   Gaussian noise in Fitness, Multi-modal, Huge number of local optima, indeterministic
# F24:   Composite Function, Huge number of local optima, Multi-modal

benchmarks_and_bounds = [optproblems.F2, [-100,100], "F2", optproblems.F4, [-100,100], "F4", optproblems.F8, [-32,32], "F8", optproblems.F13, [-3,1], "F13", optproblems.F17, [-5,5], "F17", optproblems.F24, [-5,5], "F24"]
no_dimensions = [10, 30, 50]
pop_sizes = [5, 10, 15]
num_generations = [50, 150]
tournament_t = [2, 4]
crossover_percentage = [0.5, 0.95, 1]
mutation_rates = [0.2, 0.8, 1]
decreasing_mutation_rate = [True, False]
elites = [0, 1]

average_over_range = 5

Run configuration suite over each configuration combination.

In [3]:
run_configuration_suite = False
if run_configuration_suite:

    list_for_df = []
    for i in range(0, len(benchmarks_and_bounds), 3):
        number_of_functions = (len(benchmarks_and_bounds)/3)
        print(f"Progress: {i/3} / {number_of_functions}")
        bound = benchmarks_and_bounds[i+1]
        benchmark_name = benchmarks_and_bounds[i+2]
        for no_dimension in no_dimensions:
            print(f"\tDimension: {no_dimension}")
            benchmark = benchmarks_and_bounds[i](no_dimension)
            optimal_solution = benchmark.get_optimal_solutions()
            benchmark.evaluate(optimal_solution[0])
            optimal_minimum = optimal_solution[0].objective_values
            for pop_size in pop_sizes:
                print(f"\t\tPopulation: {pop_size}")
                for t in tournament_t:
                    for crossover in crossover_percentage:
                        for mutation in mutation_rates:
                            for dmr in decreasing_mutation_rate:
                                for elite in elites:
                                    for generations in num_generations:
                                        average_fitness = 0
                                        average_difference_from_optimal = 0

                                        best_fitness = 0
                                        best_time = 0
                                        best_difference_from_optimal = 0

                                        for j in range(average_over_range):
                                            algorithm = GA(benchmark, no_dimension, bound, pop_size, num_generations=generations, t=t, crossover_percentage=crossover, mutation_rate=mutation, decreasing_mutation_rate=dmr, elite=elite)
                                            result, time = algorithm.run()
                                            fitness = result.fitness
                                            diff_from_optimal = fitness - optimal_minimum

                                            average_fitness += 1/average_over_range * fitness
                                            average_difference_from_optimal += 1/average_over_range * diff_from_optimal

                                            if j == 0:
                                                best_fitness = fitness
                                                best_difference_from_optimal = diff_from_optimal
                                                best_time = time
                                                continue

                                            if fitness < best_fitness:
                                                best_fitness = fitness

                                            if diff_from_optimal < best_difference_from_optimal:
                                                best_difference_from_optimal = diff_from_optimal

                                            if time < best_time:
                                                best_time = time

                                        result_and_configuration = [benchmark_name, optimal_minimum, best_fitness, best_difference_from_optimal, average_fitness, average_difference_from_optimal, best_time, generations, no_dimension, pop_size, t, crossover, mutation, dmr, elite, bound, average_over_range]
                                        list_for_df.append(result_and_configuration)

    labels = ["TestFunction", "OptimalMinimum", "BestFitness", "BestDiffFromOpt", "AverageFitness", "AverageDiffFromOpt", "TimePerRun", "Generations", "NoDimension", "PopSize", "TForTournament", "CrossoverPerc", "MutationRate", "DecreasingMutationRate", "Elite", "Bounds", "AverageOverRange"]
    df = pandas.DataFrame(data=list_for_df, columns=labels)
    df.to_pickle("GA_df.pkl")
else:
    df = pd.read_pickle("GA_df.pkl")

## Results of the 1st Iteration

# F2:

- Best Diff: 312.11 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 100% CR / 20% MR / DMR True / 1 Elite
    -  Error: 312.11 / 450 -> 69.36%
- Avr. Diff: 913.20 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95%  CR / 20% MR / DMR True / 1 Elite
    - Error: 913.20 / 450 -> 202.93%

In [4]:
# Best Fitness
df_F2_1st = df.query("TestFunction == 'F2'")
df_F2_1st[df_F2_1st.BestDiffFromOpt == df_F2_1st.BestDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
339,F2,-450.0,-137.88671,312.11329,1386.667757,1836.667757,0.07543,150,10,15,2,1.0,0.2,True,1,"[-100, 100]",5


In [5]:
# Best Average Fitness
df_F2_1st[df_F2_1st.AverageDiffFromOpt == df_F2_1st.AverageDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
387,F2,-450.0,136.349155,586.349155,463.202679,913.202679,0.07794,150,10,15,4,0.95,0.2,True,1,"[-100, 100]",5


In [6]:
df_F2_1st

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
0,F2,-450.0,17690.525955,18140.525955,3.213329e+04,3.258329e+04,0.011128,50,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
1,F2,-450.0,14235.020676,14685.020676,2.238459e+04,2.283459e+04,0.027806,150,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
2,F2,-450.0,6013.072511,6463.072511,1.487255e+04,1.532255e+04,0.010348,50,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
3,F2,-450.0,1868.742268,2318.742268,5.054819e+03,5.504819e+03,0.031108,150,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
4,F2,-450.0,21611.965380,22061.965380,5.134586e+04,5.179586e+04,0.009959,50,10,5,2,0.5,0.2,False,0,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,F2,-450.0,112341.304928,112791.304928,1.461942e+05,1.466442e+05,0.321514,150,50,15,4,1.0,1.0,True,1,"[-100, 100]",5
1292,F2,-450.0,590805.599163,591255.599163,9.641332e+05,9.645832e+05,0.171331,50,50,15,4,1.0,1.0,False,0,"[-100, 100]",5
1293,F2,-450.0,575018.988130,575468.988130,1.290336e+06,1.290786e+06,0.513789,150,50,15,4,1.0,1.0,False,0,"[-100, 100]",5
1294,F2,-450.0,232243.327721,232693.327721,3.293918e+05,3.298418e+05,0.154155,50,50,15,4,1.0,1.0,False,1,"[-100, 100]",5


# F4
- Best Diff: 459.44 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR False / 1 Elite
    - Error: 459.44 / 450 ->  102.09%
- Avr. Diff: 1841.4 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 95%  CR / 20% MR / DMR True / 1 Elite
    - Error: 1841.4 / 450 ->  409.2%

In [7]:
# Best Fitness
df_F4_1st = df.query("TestFunction == 'F4'")
df_F4_1st[df_F4_1st.BestDiffFromOpt == df_F4_1st.BestDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
1687,F4,-450.0,9.440808,459.440808,2122.914216,2572.914216,0.089259,150,10,15,4,0.95,0.2,False,1,"[-100, 100]",5


In [8]:
# Best Average Fitness
df_F4_1st[df_F4_1st.AverageDiffFromOpt == df_F4_1st.AverageDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
1611,F4,-450.0,553.466922,1003.466922,1391.481705,1841.481705,0.079499,150,10,15,2,0.95,0.2,True,1,"[-100, 100]",5


In [9]:
df_F4_1st

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
1296,F4,-450.0,19648.328195,20098.328195,4.668801e+04,4.713801e+04,0.009338,50,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
1297,F4,-450.0,43889.355029,44339.355029,5.329062e+04,5.374062e+04,0.028457,150,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
1298,F4,-450.0,3226.647401,3676.647401,1.438899e+04,1.483899e+04,0.009620,50,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
1299,F4,-450.0,1952.865879,2402.865879,6.795449e+03,7.245449e+03,0.032871,150,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
1300,F4,-450.0,53798.825239,54248.825239,1.240192e+05,1.244692e+05,0.011265,50,10,5,2,0.5,0.2,False,0,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,F4,-450.0,161980.060540,162430.060540,2.346768e+05,2.351268e+05,0.328651,150,50,15,4,1.0,1.0,True,1,"[-100, 100]",5
2588,F4,-450.0,404325.050245,404775.050245,1.207540e+06,1.207990e+06,0.172086,50,50,15,4,1.0,1.0,False,0,"[-100, 100]",5
2589,F4,-450.0,474857.028769,475307.028769,1.385936e+06,1.386386e+06,0.524369,150,50,15,4,1.0,1.0,False,0,"[-100, 100]",5
2590,F4,-450.0,367568.143286,368018.143286,5.447924e+05,5.452424e+05,0.155512,50,50,15,4,1.0,1.0,False,1,"[-100, 100]",5


# F8
- Best Diff: 20.17 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 80% MR / DMR True / 1 Elite
    - Error: 20.17 / 140 -> 14.4%
- Avr. Diff: 20.47 -> Same
    - Error: 20.47 / 140 -> 14.6%

In [10]:
# Best Fitness
df_F8_1st = df.query("TestFunction == 'F8'")
df_F8_1st[df_F8_1st.BestDiffFromOpt == df_F8_1st.BestDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
3011,F8,-140.0,-119.829659,20.170341,-119.524006,20.475994,0.116559,150,10,15,4,1.0,0.8,True,1,"[-32, 32]",5


In [11]:
# Best Average Fitness
df_F8_1st[df_F8_1st.AverageDiffFromOpt == df_F8_1st.AverageDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
3011,F8,-140.0,-119.829659,20.170341,-119.524006,20.475994,0.116559,150,10,15,4,1.0,0.8,True,1,"[-32, 32]",5


In [12]:
df_F8_1st

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
2592,F8,-140.0,-118.828332,21.171668,-118.601792,21.398208,0.011346,50,10,5,2,0.5,0.2,True,0,"[-32, 32]",5
2593,F8,-140.0,-118.780497,21.219503,-118.632335,21.367665,0.034004,150,10,5,2,0.5,0.2,True,0,"[-32, 32]",5
2594,F8,-140.0,-119.053467,20.946533,-118.934699,21.065301,0.015518,50,10,5,2,0.5,0.2,True,1,"[-32, 32]",5
2595,F8,-140.0,-119.263744,20.736256,-119.165199,20.834801,0.046569,150,10,5,2,0.5,0.2,True,1,"[-32, 32]",5
2596,F8,-140.0,-118.832553,21.167447,-118.571401,21.428599,0.017156,50,10,5,2,0.5,0.2,False,0,"[-32, 32]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,F8,-140.0,-118.752831,21.247169,-118.683254,21.316746,0.383418,150,50,15,4,1.0,1.0,True,1,"[-32, 32]",5
3884,F8,-140.0,-118.536631,21.463369,-118.443720,21.556280,0.194784,50,50,15,4,1.0,1.0,False,0,"[-32, 32]",5
3885,F8,-140.0,-118.496857,21.503143,-118.436180,21.563820,0.586340,150,50,15,4,1.0,1.0,False,0,"[-32, 32]",5
3886,F8,-140.0,-118.684637,21.315363,-118.661321,21.338679,0.175834,50,50,15,4,1.0,1.0,False,1,"[-32, 32]",5


# F13
- Best Diff: 1.0829 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 1.0829 / 130 ->  0.83%
- Avr. Diff: 1.5791 -> Same
    - Error: 1.5791 / 130 ->  1.21%


In [13]:
# Best Fitness
df_F13_1st = df.query("TestFunction == 'F13'")
df_F13_1st[df_F13_1st.BestDiffFromOpt == df_F13_1st.BestDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
411,F13,-130.0,-128.917016,1.082984,-128.420892,1.579108,0.130163,150,10,15,4,1.0,0.2,True,1,"[-3, 1]",5


In [14]:
# Best Average Fitness
df_F13_1st[df_F13_1st.AverageDiffFromOpt == df_F13_1st.AverageDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
411,F13,-130.0,-128.917016,1.082984,-128.420892,1.579108,0.130163,150,10,15,4,1.0,0.2,True,1,"[-3, 1]",5


In [15]:
df_F13_1st

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
0,F13,-130.0,-114.747967,15.252033,123.382008,253.382008,0.015493,50,10,5,2,0.5,0.2,True,0,"[-3, 1]",5
1,F13,-130.0,-112.484813,17.515187,-81.314126,48.685874,0.047154,150,10,5,2,0.5,0.2,True,0,"[-3, 1]",5
2,F13,-130.0,-119.482405,10.517595,-117.122011,12.877989,0.014168,50,10,5,2,0.5,0.2,True,1,"[-3, 1]",5
3,F13,-130.0,-127.111143,2.888857,-122.551732,7.448268,0.043210,150,10,5,2,0.5,0.2,True,1,"[-3, 1]",5
4,F13,-130.0,68.014676,198.014676,1145.731194,1275.731194,0.015538,50,10,5,2,0.5,0.2,False,0,"[-3, 1]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,F13,-130.0,18.446939,148.446939,86.802009,216.802009,0.559082,150,50,15,4,1.0,1.0,True,1,"[-3, 1]",5
1292,F13,-130.0,3078.646003,3208.646003,5533.162406,5663.162406,0.246466,50,50,15,4,1.0,1.0,False,0,"[-3, 1]",5
1293,F13,-130.0,5464.382972,5594.382972,9402.327325,9532.327325,0.757659,150,50,15,4,1.0,1.0,False,0,"[-3, 1]",5
1294,F13,-130.0,1636.088684,1766.088684,2376.984034,2506.984034,0.231453,50,50,15,4,1.0,1.0,False,1,"[-3, 1]",5


# F17
- Best Diff: 177.54 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 50% CR / 80% MR / DMR True / 1 Elite
    - Error: 177.54 / 120 ->  147.95%
- Avr. Diff: 223.09 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 223.09 / 120 ->  185.9%

In [16]:
# Best Fitness
df_F17_1st = df.query("TestFunction == 'F17'")
df_F17_1st[df_F17_1st.BestDiffFromOpt == df_F17_1st.BestDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
1595,F17,120.0,297.548261,177.548261,444.233114,324.233114,0.994805,150,10,15,2,0.5,0.8,True,1,"[-5, 5]",5


In [17]:
# Best Average Fitness
df_F17_1st[df_F17_1st.AverageDiffFromOpt == df_F17_1st.AverageDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
1707,F17,120.0,325.36866,205.36866,343.091339,223.091339,0.967983,150,10,15,4,1.0,0.2,True,1,"[-5, 5]",5


In [18]:
df_F17_1st

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
1296,F17,120.0,748.766137,628.766137,1079.084525,959.084525,0.113645,50,10,5,2,0.5,0.2,True,0,"[-5, 5]",5
1297,F17,120.0,591.077512,471.077512,951.976583,831.976583,0.335133,150,10,5,2,0.5,0.2,True,0,"[-5, 5]",5
1298,F17,120.0,454.579857,334.579857,557.498490,437.498490,0.110451,50,10,5,2,0.5,0.2,True,1,"[-5, 5]",5
1299,F17,120.0,403.606848,283.606848,495.791958,375.791958,0.332959,150,10,5,2,0.5,0.2,True,1,"[-5, 5]",5
1300,F17,120.0,1369.054293,1249.054293,1429.394142,1309.394142,0.110137,50,10,5,2,0.5,0.2,False,0,"[-5, 5]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,F17,120.0,692.699934,572.699934,825.222940,705.222940,4.463455,150,50,15,4,1.0,1.0,True,1,"[-5, 5]",5
2588,F17,120.0,1726.874128,1606.874128,1875.775029,1755.775029,1.468171,50,50,15,4,1.0,1.0,False,0,"[-5, 5]",5
2589,F17,120.0,1809.362841,1689.362841,1871.932725,1751.932725,4.641679,150,50,15,4,1.0,1.0,False,0,"[-5, 5]",5
2590,F17,120.0,1286.370225,1166.370225,1564.226187,1444.226187,1.467616,50,50,15,4,1.0,1.0,False,1,"[-5, 5]",5


# F24
- Best Diff: 242.76 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR False / 1 Elite
    - Error: 242.76 / 260 -> 93.36%
- Avr. Diff: 351.24 -> 150 Generation / 10 Dimension / 10 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 351.24 / 260 -> 135.09%

In [19]:
# Best Fitness
df_F24_1st = df.query("TestFunction == 'F24'")
df_F24_1st[df_F24_1st.BestDiffFromOpt == df_F24_1st.BestDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
387,F24,260.0,502.765087,242.765087,852.201586,592.201586,0.871322,150,10,15,4,0.95,0.2,True,1,"[-5, 5]",5


In [20]:
# Best Average Fitness
df_F24_1st[df_F24_1st.AverageDiffFromOpt == df_F24_1st.AverageDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
243,F24,260.0,528.283537,268.283537,611.240098,351.240098,0.597121,150,10,10,4,0.95,0.2,True,1,"[-5, 5]",5


In [21]:
df_F24_1st

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
0,F24,260.0,1581.379194,1321.379194,1772.169403,1512.169403,0.115887,50,10,5,2,0.5,0.2,True,0,"[-5, 5]",5
1,F24,260.0,1716.297855,1456.297855,1859.331639,1599.331639,0.334574,150,10,5,2,0.5,0.2,True,0,"[-5, 5]",5
2,F24,260.0,1257.358857,997.358857,1496.301694,1236.301694,0.095941,50,10,5,2,0.5,0.2,True,1,"[-5, 5]",5
3,F24,260.0,1009.196942,749.196942,1260.036581,1000.036581,0.290794,150,10,5,2,0.5,0.2,True,1,"[-5, 5]",5
4,F24,260.0,1706.802306,1446.802306,1905.026391,1645.026391,0.097299,50,10,5,2,0.5,0.2,False,0,"[-5, 5]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,F24,260.0,1604.662489,1344.662489,1633.768568,1373.768568,3.663990,150,50,15,4,1.0,1.0,True,1,"[-5, 5]",5
1292,F24,260.0,2013.695892,1753.695892,2032.972450,1772.972450,1.252370,50,50,15,4,1.0,1.0,False,0,"[-5, 5]",5
1293,F24,260.0,2022.287128,1762.287128,2078.779992,1818.779992,3.872165,150,50,15,4,1.0,1.0,False,0,"[-5, 5]",5
1294,F24,260.0,1880.468349,1620.468349,1907.107345,1647.107345,1.220936,50,50,15,4,1.0,1.0,False,1,"[-5, 5]",5


# Conclusion:

The optimizer performed the best on F8 and F13. F13 error was the lowest with the best fitness 0.38% away from the optimal. F8's best fitness was 14.4% away from the optimal. While the F13 error is good F8 is a significantly larger from the optimal and can be improved. The GA performed poorly on the rest of the test functions. The best fitness errors are as follows:
- F2:  69.36%
- F4: 102.09%
- F17: 147.95%
- F24: 93.36%

F8 and F13 share a similarity with their best scores. Both test function used a similar test configuration to achieve the best scores. `150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / DMR True / 1 Elite`. The only difference is that F8 used 80% Mutation Rate whereas for F13 the best score was achieved with 20%.

Each test function scored the best after 150 generations, so we decided to increase the number of generations to see how they improve. For the 2nd iteration we kept the configuration of the best settings per test function. Each test function has two best score. Best fitness and best average fitness. The parameters with low frequency from the bests weren't carried on to the 2nd iteration if the parameter comes from the best configuration with huge error. I.e. DMR false setting wasn't used as it's from F24 with high error.

Frequency of configuration among best difference and average difference from optimal:
- Generation:
    - 50 -> 0
    - 150 -> 12
- Dimension:
    - 10 -> 12
    - 30 -> 0
    - 50 -> 0
- Population Size:
    - 5 -> 0
    - 10 -> 1
    - 15 -> 11
- Tournament Size:
    - 2 -> 3
    - 4 -> 9
- Crossover Rate:
    - 50% -> 1
    - 95% -> 5
    - 100% -> 6
- Mutation Rate:
    - 20% -> 9
    - 80% -> 3
    - 100% -> 0
- Decreasing Mutation Rate:
    - True -> 10
    - False -> 2
- Elites:
    - 0 -> 0
    - 1 -> 12

# 2nd Iteration of Configurations
Same algorithm was used as for the 1st iteration but different hyper-parameters. Increased the number of generations to 250, 500, 1000. The population size was increased too and some configurations weren't carried over from iteration 1, i.e. the number of dimension was reduced to only 10, the False setting for DMR wasn't used as the error was high for the results in the 1st iteration which had false.

In [22]:
benchmarks_and_bounds = [optproblems.F2, [-100,100], "F2", optproblems.F4, [-100,100], "F4", optproblems.F8, [-32,32], "F8", optproblems.F13, [-3,1], "F13", optproblems.F17, [-5,5], "F17", optproblems.F24, [-5,5], "F24"]
no_dimensions = [10]
num_generations = [250, 500, 1000]
pop_sizes = [10, 15, 20]
tournament_t = [2, 4]
crossover_percentage = [0.95, 1]
mutation_rates = [0.2, 0.8]
decreasing_mutation_rate = [True]
elites = [0, 1]

average_over_range = 5

In [23]:
run_configuration_suite = False
if run_configuration_suite:

    list_for_df = []
    for i in range(0, len(benchmarks_and_bounds), 3):
        number_of_functions = (len(benchmarks_and_bounds)/3)
        print(f"Progress: {i/3} / {number_of_functions}")
        bound = benchmarks_and_bounds[i+1]
        benchmark_name = benchmarks_and_bounds[i+2]
        for no_dimension in no_dimensions:
            print(f"\tDimension: {no_dimension}")
            benchmark = benchmarks_and_bounds[i](no_dimension)
            optimal_solution = benchmark.get_optimal_solutions()
            benchmark.evaluate(optimal_solution[0])
            optimal_minimum = optimal_solution[0].objective_values
            for pop_size in pop_sizes:
                print(f"\t\tPopulation: {pop_size}")
                for t in tournament_t:
                    for crossover in crossover_percentage:
                        for mutation in mutation_rates:
                            for dmr in decreasing_mutation_rate:
                                for elite in elites:
                                    for generations in num_generations:
                                        average_fitness = 0
                                        average_difference_from_optimal = 0

                                        best_fitness = 0
                                        best_time = 0
                                        best_difference_from_optimal = 0

                                        for j in range(average_over_range):
                                            algorithm = GA(benchmark, no_dimension, bound, pop_size, num_generations=generations, t=t, crossover_percentage=crossover, mutation_rate=mutation, decreasing_mutation_rate=dmr, elite=elite)
                                            result, time = algorithm.run()
                                            fitness = result.fitness
                                            diff_from_optimal = fitness - optimal_minimum

                                            average_fitness += 1/average_over_range * fitness
                                            average_difference_from_optimal += 1/average_over_range * diff_from_optimal

                                            if j == 0:
                                                best_fitness = fitness
                                                best_difference_from_optimal = diff_from_optimal
                                                best_time = time
                                                continue

                                            if fitness < best_fitness:
                                                best_fitness = fitness

                                            if diff_from_optimal < best_difference_from_optimal:
                                                best_difference_from_optimal = diff_from_optimal

                                            if time < best_time:
                                                best_time = time

                                        result_and_configuration = [benchmark_name, optimal_minimum, best_fitness, best_difference_from_optimal, average_fitness, average_difference_from_optimal, best_time, generations, no_dimension, pop_size, t, crossover, mutation, dmr, elite, bound, average_over_range]
                                        list_for_df.append(result_and_configuration)

    labels = ["TestFunction", "OptimalMinimum", "BestFitness", "BestDiffFromOpt", "AverageFitness", "AverageDiffFromOpt", "TimePerRun", "Generations", "NoDimension", "PopSize", "TForTournament", "CrossoverPerc", "MutationRate", "DecreasingMutationRate", "Elite", "Bounds", "AverageOverRange"]
    df_II = pandas.DataFrame(data=list_for_df, columns=labels)
    df_II.to_pickle("GA_df_specific.pkl")
else:
    df_II = pd.read_pickle("GA_df_specific.pkl")

# Results of the 2nd Iteration

# F2
### For 1st Iteration:
- Best Diff: 312.11 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 100% CR / 20% MR / DMR True / 1 Elite
    -  Error: 312.11 / 450 -> 69.36%
- Avr. Diff: 913.20 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95%  CR / 20% MR / DMR True / 1 Elite
    - Error: 913.20 / 450 -> 202.93%

### For 2nd Iteration:
- Best Diff: 33.482 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 0 Elite
    -  Error: 33.482 / 450 -> 7.4%
- Avr. Diff: 113.75 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 113.75 / 450 -> 25.27%

## Conclusion for F2:
- Significant improvement by increasing the generations from 150 -> 1000 and the population from 15 -> 20.
- Best diff error is ~10× less
- Avg diff error is ~8× less
- In the 2nd iteration the best fitness was achieved by 4 as tournament size and 95% for crossover rate instead of 2 and 100%.

In [24]:
df_II.query("TestFunction == 'F2'").sort_values(by=["BestDiffFromOpt"])

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
122,F2,-450.0,-416.517369,33.482631,-105.535665,344.464335,0.678949,1000,10,20,4,0.95,0.2,True,0,"[-100, 100]",5
137,F2,-450.0,-408.762250,41.237750,-292.125138,157.874862,0.716443,1000,10,20,4,1.00,0.2,True,1,"[-100, 100]",5
113,F2,-450.0,-397.671236,52.328764,-228.550850,221.449150,0.686052,1000,10,20,2,1.00,0.2,True,1,"[-100, 100]",5
125,F2,-450.0,-391.244262,58.755738,-336.241540,113.758460,0.702944,1000,10,20,4,0.95,0.2,True,1,"[-100, 100]",5
77,F2,-450.0,-385.323374,64.676626,-141.952623,308.047377,0.523073,1000,10,15,4,0.95,0.2,True,1,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,F2,-450.0,4946.517936,5396.517936,7945.357578,8395.357578,0.315453,500,10,15,2,0.95,0.8,True,0,"[-100, 100]",5
12,F2,-450.0,5026.711628,5476.711628,11279.574303,11729.574303,0.084131,250,10,10,2,1.00,0.2,True,0,"[-100, 100]",5
102,F2,-450.0,5634.522811,6084.522811,8568.824344,9018.824344,0.202083,250,10,20,2,0.95,0.8,True,0,"[-100, 100]",5
19,F2,-450.0,5651.558023,6101.558023,13316.792343,13766.792343,0.207827,500,10,10,2,1.00,0.8,True,0,"[-100, 100]",5


In [25]:
df_II.query("TestFunction == 'F2'").sort_values(by=["AverageDiffFromOpt"])

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
125,F2,-450.0,-391.244262,58.755738,-336.241540,113.758460,0.702944,1000,10,20,4,0.95,0.2,True,1,"[-100, 100]",5
137,F2,-450.0,-408.762250,41.237750,-292.125138,157.874862,0.716443,1000,10,20,4,1.00,0.2,True,1,"[-100, 100]",5
101,F2,-450.0,-354.126824,95.873176,-260.114914,189.885086,0.676758,1000,10,20,2,0.95,0.2,True,1,"[-100, 100]",5
113,F2,-450.0,-397.671236,52.328764,-228.550850,221.449150,0.686052,1000,10,20,2,1.00,0.2,True,1,"[-100, 100]",5
124,F2,-450.0,-307.048301,142.951699,-216.653180,233.346820,0.351216,500,10,20,4,0.95,0.2,True,1,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,F2,-450.0,3300.153212,3750.153212,10927.162940,11377.162940,0.418171,1000,10,10,2,1.00,0.8,True,0,"[-100, 100]",5
12,F2,-450.0,5026.711628,5476.711628,11279.574303,11729.574303,0.084131,250,10,10,2,1.00,0.2,True,0,"[-100, 100]",5
6,F2,-450.0,5714.207052,6164.207052,11293.868002,11743.868002,0.101580,250,10,10,2,0.95,0.8,True,0,"[-100, 100]",5
18,F2,-450.0,4489.186327,4939.186327,11464.228269,11914.228269,0.104034,250,10,10,2,1.00,0.8,True,0,"[-100, 100]",5


# F4
### For 1st Iteration
- Best Diff: 459.44 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR False / 1 Elite
    - Error: 459.44 / 450 ->  102.09%
- Avr. Diff: 1841.4 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 95%  CR / 20% MR / DMR True / 1 Elite
    - Error: 1841.4 / 450 ->  409.2%

### For 2nd Iteration:
- Best Diff: 56.63 -> 500 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 56.63 / 450 -> 12.58%
- Avr. Diff: 190.78 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 190.78 / 450 -> 42.39%

## Conclusion for F4:
- Significant change by increasing the generation.
- The best fitness was achieved with 500 generation instead of 1000 and the population size was same for both iterations.
- The best fitness parameters didn't change much. Decreasing MR was used and the generation was increased from 150 to 500. However, for the average fitness the parameters changed. Higher population (20), generation (1000), tournament size (4) and crossover rate (100%) was used.

In [26]:
df_II.query("TestFunction == 'F4'").sort_values(by=["BestDiffFromOpt"])

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
220,F4,-450.0,-393.367303,56.632697,-22.736756,427.263244,0.274398,500,10,15,4,0.95,0.2,True,1,"[-100, 100]",5
269,F4,-450.0,-370.534131,79.465869,-171.885283,278.114717,0.754785,1000,10,20,4,0.95,0.2,True,1,"[-100, 100]",5
245,F4,-450.0,-365.166254,84.833746,-140.894787,309.105213,0.719845,1000,10,20,2,0.95,0.2,True,1,"[-100, 100]",5
287,F4,-450.0,-349.955701,100.044299,-130.511791,319.488209,0.918671,1000,10,20,4,1.00,0.8,True,1,"[-100, 100]",5
221,F4,-450.0,-343.758517,106.241483,-168.488252,281.511748,0.571917,1000,10,15,4,0.95,0.2,True,1,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,F4,-450.0,6337.334092,6787.334092,8872.264287,9322.264287,0.113453,250,10,10,4,1.00,0.8,True,0,"[-100, 100]",5
198,F4,-450.0,6339.920623,6789.920623,14491.071605,14941.071605,0.167055,250,10,15,2,0.95,0.8,True,0,"[-100, 100]",5
150,F4,-450.0,6914.098834,7364.098834,13935.881487,14385.881487,0.109907,250,10,10,2,0.95,0.8,True,0,"[-100, 100]",5
164,F4,-450.0,7075.860228,7525.860228,13371.260980,13821.260980,0.436771,1000,10,10,2,1.00,0.8,True,0,"[-100, 100]",5


In [27]:
df_II.query("TestFunction == 'F4'").sort_values(by=["AverageDiffFromOpt"])

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
281,F4,-450.0,-313.430371,136.569629,-259.211958,190.788042,0.756325,1000,10,20,4,1.00,0.2,True,1,"[-100, 100]",5
280,F4,-450.0,-304.385641,145.614359,-218.041289,231.958711,0.377391,500,10,20,4,1.00,0.2,True,1,"[-100, 100]",5
185,F4,-450.0,-318.794511,131.205489,-178.949035,271.050965,0.405079,1000,10,10,4,1.00,0.2,True,1,"[-100, 100]",5
269,F4,-450.0,-370.534131,79.465869,-171.885283,278.114717,0.754785,1000,10,20,4,0.95,0.2,True,1,"[-100, 100]",5
221,F4,-450.0,-343.758517,106.241483,-168.488252,281.511748,0.571917,1000,10,15,4,0.95,0.2,True,1,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,F4,-450.0,6149.657651,6599.657651,12196.537572,12646.537572,0.110522,250,10,10,2,1.00,0.8,True,0,"[-100, 100]",5
164,F4,-450.0,7075.860228,7525.860228,13371.260980,13821.260980,0.436771,1000,10,10,2,1.00,0.8,True,0,"[-100, 100]",5
150,F4,-450.0,6914.098834,7364.098834,13935.881487,14385.881487,0.109907,250,10,10,2,0.95,0.8,True,0,"[-100, 100]",5
198,F4,-450.0,6339.920623,6789.920623,14491.071605,14941.071605,0.167055,250,10,15,2,0.95,0.8,True,0,"[-100, 100]",5


# F8
(2nd best fitness among the first iteration)
### For 1st Iteration:
- Best Diff: 20.17 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 80% MR / DMR True / 1 Elite
    - Error: 20.17 / 140 -> 14.4%
- Avr. Diff: 20.47 -> Same
    - Error: 20.47 / 140 -> 14.6%

### For 2nd Iteration:
- Best Diff: 20.29 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 0 Elite
    - Error: 20.29 / 140 -> 14.49%
- Avr. Diff: 20.41 -> 1000 Generation / 10 Dimension / 10 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 0 Elite
    - Error: 20.41 / 140 -> 14.57%

## Conclusion for F8:
- Best fitness error slightly increased!!!
- Gen 500 had similar result for best fitness as 1000.
- Average fitness improved slightly.
- No significant changes achieved when running it on higher number, as for previous functions.
- Since best fitness was reached at the 150 generation we conclude that for F8 the optimizer works well as for other test functions it took 500, 1000 generations to reach similar results.
- Still the second best evaluated fitness function as in the first iteration.

In [28]:
df_II.query("TestFunction == 'F8'").sort_values(by=["BestDiffFromOpt"])

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
422,F8,-140.0,-119.709474,20.290526,-119.389335,20.610665,0.866229,1000,10,20,4,1.00,0.2,True,0,"[-32, 32]",5
424,F8,-140.0,-119.706343,20.293657,-119.474575,20.525425,0.439828,500,10,20,4,1.00,0.2,True,1,"[-32, 32]",5
407,F8,-140.0,-119.702463,20.297537,-119.548319,20.451681,1.021265,1000,10,20,2,1.00,0.8,True,1,"[-32, 32]",5
353,F8,-140.0,-119.685671,20.314329,-119.482964,20.517036,0.635577,1000,10,15,2,1.00,0.2,True,1,"[-32, 32]",5
323,F8,-140.0,-119.684831,20.315169,-119.495108,20.504892,0.536117,1000,10,10,4,0.95,0.8,True,1,"[-32, 32]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,F8,-140.0,-118.995207,21.004793,-118.875894,21.124106,0.244061,250,10,20,2,0.95,0.8,True,0,"[-32, 32]",5
288,F8,-140.0,-118.968130,21.031870,-118.857024,21.142976,0.114117,250,10,10,2,0.95,0.2,True,0,"[-32, 32]",5
294,F8,-140.0,-118.946998,21.053002,-118.850106,21.149894,0.124909,250,10,10,2,0.95,0.8,True,0,"[-32, 32]",5
343,F8,-140.0,-118.946147,21.053853,-118.846371,21.153629,0.383594,500,10,15,2,0.95,0.8,True,0,"[-32, 32]",5


In [29]:
df_II.query("TestFunction == 'F8'").sort_values(by=["AverageDiffFromOpt"])

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
329,F8,-140.0,-119.658589,20.341411,-119.583318,20.416682,0.472398,1000,10,10,4,1.00,0.2,True,1,"[-32, 32]",5
419,F8,-140.0,-119.662022,20.337978,-119.565211,20.434789,1.047593,1000,10,20,4,0.95,0.8,True,1,"[-32, 32]",5
407,F8,-140.0,-119.702463,20.297537,-119.548319,20.451681,1.021265,1000,10,20,2,1.00,0.8,True,1,"[-32, 32]",5
395,F8,-140.0,-119.620632,20.379368,-119.539180,20.460820,1.011709,1000,10,20,2,0.95,0.8,True,1,"[-32, 32]",5
425,F8,-140.0,-119.605031,20.394969,-119.530749,20.469251,0.929400,1000,10,20,4,1.00,0.2,True,1,"[-32, 32]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,F8,-140.0,-119.060413,20.939587,-118.855719,21.144281,0.128400,250,10,10,4,0.95,0.8,True,0,"[-32, 32]",5
294,F8,-140.0,-118.946998,21.053002,-118.850106,21.149894,0.124909,250,10,10,2,0.95,0.8,True,0,"[-32, 32]",5
343,F8,-140.0,-118.946147,21.053853,-118.846371,21.153629,0.383594,500,10,15,2,0.95,0.8,True,0,"[-32, 32]",5
354,F8,-140.0,-118.891031,21.108969,-118.748037,21.251963,0.191671,250,10,15,2,1.00,0.8,True,0,"[-32, 32]",5


# F13
(Best fitness among the first iteration)
### For 1st Iteration:
- Best Diff: 1.0829 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 1.0829 / 130 ->  0.83%
- Avr. Diff: 1.5791 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 1.5791 / 130 ->  1.21%

### For 2nd Iteration:
- Best Diff: 0.3448 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 0.3448 / 130 ->  0.26%
- Avr. Diff: 0.6613 -> 1000 Generation / 15 Dimension / 20 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
   - Error: 0.6613 / 130 ->  0.5%

## Conclusion:
- Significant improvement by increasing the population size and generations.
- Still the best fitness.
- Best fitness is 3× less.
- Average fitness is ~2.5× less.
- The configuration is mostly the same.


In [30]:
df_II.query("TestFunction == 'F13'").sort_values(by=["BestDiffFromOpt"])

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
557,F13,-130.0,-129.655200,0.344800,-129.314413,0.685587,1.201319,1000,10,20,4,0.95,0.2,True,1,"[-3, 1]",5
569,F13,-130.0,-129.525787,0.474213,-129.338676,0.661324,1.187343,1000,10,20,4,1.00,0.2,True,1,"[-3, 1]",5
521,F13,-130.0,-129.496902,0.503098,-129.280903,0.719097,0.863228,1000,10,15,4,1.00,0.2,True,1,"[-3, 1]",5
566,F13,-130.0,-129.495214,0.504786,-129.122606,0.877394,1.182098,1000,10,20,4,1.00,0.2,True,0,"[-3, 1]",5
509,F13,-130.0,-129.481843,0.518157,-129.170962,0.829038,0.869595,1000,10,15,4,0.95,0.2,True,1,"[-3, 1]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,F13,-130.0,-124.714676,5.285324,-123.254012,6.745988,0.242735,250,10,15,2,1.00,0.8,True,0,"[-3, 1]",5
450,F13,-130.0,-124.620314,5.379686,-122.238652,7.761348,0.162304,250,10,10,2,1.00,0.8,True,0,"[-3, 1]",5
486,F13,-130.0,-124.147934,5.852066,-121.811208,8.188792,0.247298,250,10,15,2,0.95,0.8,True,0,"[-3, 1]",5
534,F13,-130.0,-123.533443,6.466557,-121.386809,8.613191,0.323671,250,10,20,2,0.95,0.8,True,0,"[-3, 1]",5


In [31]:
df_II.query("TestFunction == 'F13'").sort_values(by=["AverageDiffFromOpt"])

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
569,F13,-130.0,-129.525787,0.474213,-129.338676,0.661324,1.187343,1000,10,20,4,1.00,0.2,True,1,"[-3, 1]",5
557,F13,-130.0,-129.655200,0.344800,-129.314413,0.685587,1.201319,1000,10,20,4,0.95,0.2,True,1,"[-3, 1]",5
533,F13,-130.0,-129.374198,0.625802,-129.281827,0.718173,1.152005,1000,10,20,2,0.95,0.2,True,1,"[-3, 1]",5
521,F13,-130.0,-129.496902,0.503098,-129.280903,0.719097,0.863228,1000,10,15,4,1.00,0.2,True,1,"[-3, 1]",5
568,F13,-130.0,-129.360961,0.639039,-129.220925,0.779075,0.593360,500,10,20,4,1.00,0.2,True,1,"[-3, 1]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,F13,-130.0,-124.841950,5.158050,-122.705334,7.294666,0.319519,500,10,10,2,1.00,0.8,True,0,"[-3, 1]",5
450,F13,-130.0,-124.620314,5.379686,-122.238652,7.761348,0.162304,250,10,10,2,1.00,0.8,True,0,"[-3, 1]",5
486,F13,-130.0,-124.147934,5.852066,-121.811208,8.188792,0.247298,250,10,15,2,0.95,0.8,True,0,"[-3, 1]",5
534,F13,-130.0,-123.533443,6.466557,-121.386809,8.613191,0.323671,250,10,20,2,0.95,0.8,True,0,"[-3, 1]",5


# F17
### For 1st Iteration:
- Best Diff: 177.54 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 50% CR / 80% MR / DMR True / 1 Elite
    - Error: 177.54 / 120 ->  147.95%
- Avr. Diff: 223.09 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 223.09 / 120 ->  185.9%

### For 2nd Iteration:
- Best Diff: 135.67 -> 1000 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 135.67 / 120 ->  113.05%
- Avr. Diff: 169.43 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 169.43 / 120 ->  141.19%

## Conclusion:
- Some improvement but the results are still off. Seems like the algorithm is stuck in a local minimum.
- The algorithm didn't do well on this test function with the configurations from the 1st and 2nd iteration.



In [32]:
df_II.query("TestFunction == 'F17'").sort_values(by=["BestDiffFromOpt"])

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
665,F17,120.0,255.678788,135.678788,309.474039,189.474039,6.625972,1000,10,15,4,1.00,0.2,True,1,"[-5, 5]",5
653,F17,120.0,257.460432,137.460432,308.726453,188.726453,6.658226,1000,10,15,4,0.95,0.2,True,1,"[-5, 5]",5
701,F17,120.0,260.866183,140.866183,295.572479,175.572479,9.595452,1000,10,20,4,0.95,0.2,True,1,"[-5, 5]",5
698,F17,120.0,262.759292,142.759292,293.607191,173.607191,9.495039,1000,10,20,4,0.95,0.2,True,0,"[-5, 5]",5
710,F17,120.0,262.864028,142.864028,289.434116,169.434116,9.537566,1000,10,20,4,1.00,0.2,True,0,"[-5, 5]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,F17,120.0,426.005718,306.005718,475.061585,355.061585,3.313387,500,10,15,2,1.00,0.8,True,0,"[-5, 5]",5
630,F17,120.0,441.864608,321.864608,475.034141,355.034141,1.673728,250,10,15,2,0.95,0.8,True,0,"[-5, 5]",5
690,F17,120.0,447.952332,327.952332,468.298938,348.298938,2.380361,250,10,20,2,1.00,0.8,True,0,"[-5, 5]",5
594,F17,120.0,460.188353,340.188353,482.543281,362.543281,1.063629,250,10,10,2,1.00,0.8,True,0,"[-5, 5]",5


In [33]:
df_II.query("TestFunction == 'F17'").sort_values(by=["AverageDiffFromOpt"])

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
710,F17,120.0,262.864028,142.864028,289.434116,169.434116,9.537566,1000,10,20,4,1.00,0.2,True,0,"[-5, 5]",5
698,F17,120.0,262.759292,142.759292,293.607191,173.607191,9.495039,1000,10,20,4,0.95,0.2,True,0,"[-5, 5]",5
701,F17,120.0,260.866183,140.866183,295.572479,175.572479,9.595452,1000,10,20,4,0.95,0.2,True,1,"[-5, 5]",5
712,F17,120.0,263.244977,143.244977,296.044631,176.044631,4.710266,500,10,20,4,1.00,0.2,True,1,"[-5, 5]",5
617,F17,120.0,263.515907,143.515907,297.071366,177.071366,4.491223,1000,10,10,4,1.00,0.2,True,1,"[-5, 5]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,F17,120.0,426.005718,306.005718,475.061585,355.061585,3.313387,500,10,15,2,1.00,0.8,True,0,"[-5, 5]",5
606,F17,120.0,388.518597,268.518597,482.031197,362.031197,1.077225,250,10,10,4,0.95,0.8,True,0,"[-5, 5]",5
594,F17,120.0,460.188353,340.188353,482.543281,362.543281,1.063629,250,10,10,2,1.00,0.8,True,0,"[-5, 5]",5
678,F17,120.0,390.249467,270.249467,509.881629,389.881629,2.205039,250,10,20,2,0.95,0.8,True,0,"[-5, 5]",5


# F24
### For 1st Iteration:
- Best Diff: 242.76 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR False / 1 Elite
    - Error: 242.76 / 260 -> 93.36%
- Avr. Diff: 351.24 -> 150 Generation / 10 Dimension / 10 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 351.24 / 260 -> 135.09%

### For 2nd Iteration:
- Best Diff: 200.34 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR False / 1 Elite
    - Error: 200.34 / 260 -> 77.05%
- Avr. Diff: 324.58 -> 150 Generation / 10 Dimension / 10 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 324.58 / 260 -> 124.83%

## Conclusion:
- Similar as for F17, some improvement but the results are still off. Seems like the algorithm is stuck in a local minimum.
- The algorithm didn’t do well on this test function with the configurations from the 1st and 2nd iteration.


In [34]:
df_II.query("TestFunction == 'F24'").sort_values(by=["BestDiffFromOpt"])

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
857,F24,260.0,460.337168,200.337168,584.581858,324.581858,7.936573,1000,10,20,4,1.00,0.2,True,1,"[-5, 5]",5
761,F24,260.0,460.689128,200.689128,711.527525,451.527525,4.434302,1000,10,10,4,1.00,0.2,True,1,"[-5, 5]",5
797,F24,260.0,460.752705,200.752705,524.638335,264.638335,6.467229,1000,10,15,4,0.95,0.2,True,1,"[-5, 5]",5
856,F24,260.0,460.799368,200.799368,528.006980,268.006980,3.958159,500,10,20,4,1.00,0.2,True,1,"[-5, 5]",5
833,F24,260.0,460.868514,200.868514,463.509624,203.509624,8.511882,1000,10,20,2,1.00,0.2,True,1,"[-5, 5]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,F24,260.0,1286.260499,1026.260499,1479.115352,1219.115352,2.187214,500,10,10,2,0.95,0.8,True,0,"[-5, 5]",5
822,F24,260.0,1318.956146,1058.956146,1379.883535,1119.883535,2.159867,250,10,20,2,0.95,0.8,True,0,"[-5, 5]",5
786,F24,260.0,1350.101610,1090.101610,1498.227576,1238.227576,1.671418,250,10,15,2,1.00,0.8,True,0,"[-5, 5]",5
738,F24,260.0,1405.072487,1145.072487,1555.680928,1295.680928,1.057235,250,10,10,2,1.00,0.8,True,0,"[-5, 5]",5


In [35]:
df_II.query("TestFunction == 'F24'").sort_values(by=["AverageDiffFromOpt"])

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
833,F24,260.0,460.868514,200.868514,463.509624,203.509624,8.511882,1000,10,20,2,1.00,0.2,True,1,"[-5, 5]",5
854,F24,260.0,461.477927,201.477927,466.282080,206.282080,7.800911,1000,10,20,4,1.00,0.2,True,0,"[-5, 5]",5
842,F24,260.0,462.827914,202.827914,466.952246,206.952246,7.932765,1000,10,20,4,0.95,0.2,True,0,"[-5, 5]",5
773,F24,260.0,461.940622,201.940622,467.611685,207.611685,6.491704,1000,10,15,2,0.95,0.2,True,1,"[-5, 5]",5
806,F24,260.0,462.411960,202.411960,468.385361,208.385361,6.461562,1000,10,15,4,1.00,0.2,True,0,"[-5, 5]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,F24,260.0,1286.260499,1026.260499,1479.115352,1219.115352,2.187214,500,10,10,2,0.95,0.8,True,0,"[-5, 5]",5
786,F24,260.0,1350.101610,1090.101610,1498.227576,1238.227576,1.671418,250,10,15,2,1.00,0.8,True,0,"[-5, 5]",5
774,F24,260.0,1231.169233,971.169233,1522.416300,1262.416300,1.620513,250,10,15,2,0.95,0.8,True,0,"[-5, 5]",5
726,F24,260.0,1483.708386,1223.708386,1536.934710,1276.934710,1.068954,250,10,10,2,0.95,0.8,True,0,"[-5, 5]",5


# Conclusion
Overall, the algorithm improved by increasing the number of generations and size of the population. The greatest improvement happened on the first two uni-modal functions (F2, F4). For the 2nd iteration, there was a notable improvement in the extended multimodal function (F13). However, on this function, the optimizer did well already in the 1st iteration and the change wasn’t as enormous as it was for the two uni-modal functions. The single function (F8) didn’t improve for the 2nd iteration, the best fitness on this test function did decrease slightly. The algorithm achieved a similar result as it did with the lower generation number in the 1st iteration, which implies that we found a good solution with a low error rate and low generations. The composite functions (F17, F24) did improve too, but their error was too high, with these configurations GA couldn’t a suitable solution for the test function. The high number of local optima that these composite functions have suggests that the optimizer algorithm got stuck in a local optimum. This can be avoided by increasing the population number.